<a href="https://colab.research.google.com/github/sun-in-universe/blue_dot/blob/main/Regularization%26backdrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정규화
= 과대적합 overfitting 문제 해결

- 정규화 모드 : non-zero value에 lambd추가
"lambda"를 지 않는 이유는 "lambda"가 python keyword이기 때문
- drop out : 1보다 작은 keep_prop을 value에 setting하기

우선 아무런 regularitiaon을 해본 후, 다음과 같은 정규화 실행

- L2regularization functions : "compute_cost_with_regularizaiton()" & "backward_proopagation_with_regularization()"
- Dropout functions : "forward_propagation_with_dropout()" & "backward_propagation_with_dropout()"

In [ ]:
def model(X, Y, learning_rate=0.3, num_iterations = 30000, lamdb=0, keep_prop = 1)
  grads = {}
  costs = []
  m = X.shape[1]
  layer_dims = [X.shape[0], 20, 3, 1]

  #initialize parameter dictionary
  parameters = initialize_Xavier_parameter(layer_dims)

  #Loop (gradient descent)

  for i in range(0, num_iterations):
    #forward propagation
    if keep_prop == 1:
      a3, cache = forward_propagation(X, parameters) #3번째 a값 a3, cache=W, b
    #keep_prop < 1인 경우, 이 모델에서는 keep_prop이 1이기때문에 해당 X
    elif keep_prop < 1:
      a3, cache = forward_propagation_with_dropout(X, parameters, keep_prop)

    #cost function
    if lamdb = 0 :
      cost = compute_cost(a3, Y)
    #keep_prop와 마찬가지로 이 모델에서는 lamdb=0이기 때문에 해당 X, 아래 함수 구현
    else:
      cost = compute_cost_with_regularization(a3, Y, parameters, lambd)

    #backward propagation
    if lambd == 0 and keep_prob == 1:
      grads = backward_propagation(X, Y, cache)
    elif lambd != 0:
      grads = backward_propagation_with_regularization(X, Y, cache, lambd)
    elif keep_prob < 1:
      grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)

    #update parameters.
    parameters = update_parameters(parameters, grads, learning_rate)


## compute_cost_with_regularaization

L2_regularization_cost = (1/𝑚) * (𝜆/2) * ∑𝑘∑𝑗𝑊[𝑙]2𝑘,𝑗

cost = cross_entropy_cost + L2_regularization_cost

  

In [1]:
# compute_cost_with_regularization

def compute_cost_with_regularization(A3, Y, parameters, lambd):
  """
  implement the cost function with L2 regularization.

  Arguments:
  A3 -- post-activation, output of forward propagation
  Y -- "true" labels vector, of shape
  parameters -- python dictionary containing parameters of the model

  Returns:
  cost - value of the regularized loss function
  """

  m = Y.shape[1]
  W1 = parameters["W1"]
  W2 = parameters["W2"]
  W3 = parameters["W3"]

  cross_entropy_cost = compute_cost(A3, Y)

  L2_regularization_cost = (1 / m) * (lambd / 2) * sum(np.sum(np.square(parameters["W" + strl(l)])) for i in range(1, 4))
  cost = cross_entropy_cost + L2_regularization_cost

  return cost

##backward_propagation_with_regularization

- 역전파 정규화를 하게 되면, dW1, dW2, dW3도 변화하게 된다.

- 𝑑((1/2)* (𝜆/𝑚) * 𝑊**2)= (𝜆/𝑚) * 𝑊 ->각각의  gradients마다 더해주기

In [ ]:
#backward_propagation_with_regularization
import numpy as np

def backward_propagation_with_regularization(X,. Y, cache, lambd):
  m = X.shape[1]
  #cache를 파라미터에서 받았음에도 또 쓰는 이유는, 분해함으로써 역전파에서 사용 up
  (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

  dZ3 = A3 - Y
  dW3 = (1/m) * np.dot(dZ3, A2.T) + (lambd / m) * W3
  db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)

  dA2 = np.dot(W3.T, dZ3)
  #dA2와 np.int64를 곱한 이유는 relu함수 특성때문
  dZ2 = np.multiply(dA2, np.int64(A2>0))
  dW2 = (1/m) * np.dot(dZ2, A1.T) + (lambd / m) * W2
  db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

  dA1= np.dot(W2.T, dZ2)
  dZ1 = np.multiply(dA1, np.int64(A2 > 0))
  dW1 = (1/m) * np.dot(dZ1, X.t) + (lambd) * W1
  db1 = (1/m) * np.sum(dZ1, axis=1, keepdims= True)

  gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
               "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1,
              "dZ1": dZ1, "dW1": dW1, "db1": db1}

  return gradients

- lambd는 dev set에서 tuning할 수 있는 하이퍼파라미터다.
- L2정규화는 결정 경계를 부드럽게 해준다 = 즉, 과대 적합을 방지한다. 하지만, 람다 값이 너무 크면 oversmooth해져서, high bias 문제가 발생할 수 있다.

- L2 정규화 = 가중치를 작게 해줘서 모델을 심플하게 만들어준다. 즉, 손실함수에서 가중치의 제곱값에 벌점을줌으로써(penalty부여) 가중치를 줄일 수 있다.

- cost computation = cost에 정규화 값을 더하는 것
- backpropagation function = 가중치 행렬값의 미분값에 extra terms 추가
- Weight decay
= 가중치가 너무 작아져서 소멸되버리는것

## dropout
= 반복횟수 별로 몇몇 뉴런들을 shut down하는것

- shout down = 0으로 만들어버리기
- 1-keep_prob확률로 뉴런을 0으로 만든다.
- dropped된 뉴런들은 순전파나 역전파에 기여하지 않는다.
- 즉, dropout은 곧 각각의 반복횟수마다 서로 다른 모델을 훈련하는 것과 같다. 드롭아웃을 하게 되면, 뉴런이 덜 민감해지고 이는 곧 overfitting을 막는다!

(1) Random matrix D1 = [d1, d2 ... ]을 A1과 같은 차원으로 만든다.
(2) D1에서의 가각 요소들은 keep_prob이나 0을 가진다.
(3) for i,v in enumerate(x):
    if v < keep_prob:
        x[i] = 1
    else: # v >= keep_prob
        x[i] = 0


In [ ]:
# forward_propagation_with_dropout

def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5):
  np. random.seed(1)

  #retrive parameters
  W1 = parameters["W1"]
  b1 = parameters["b1"]
  W2 = parameters["W2"]
  b2 = parameters["b2"]
  W3 = parameters["W3"]
  b3 = parameters["b3"]

  Z1 = np.dot(W1, X) + b1
  A1 = relu(Z1)

  #step1 : initialize matrix D1
  D1 = np.random.rand(A1.shape[0], A1.shape[1])
  #step2 : covert entries of D1 to 0 or 1, astype(int): 이 부분은 불리언 값을 정수로 변환합니다. True는 1로, False는 0으로 변환됩니다.
  D1 = (D1 < keep_prob).astype(int)
  #step3 : shut down some neurons of A1
  A1 = A1 * D1
  #step4 : scale the value of neurons that haven't been shut down: 드롭아웃의 정규화 보장
  A1 = A1 / (keep_prob)

  Z2 = np.dot(W2, A1) + b2
  A2 = relu(Z2)
  D2 = np.random.rand(A2.shape[0], A2.shape[1])
  D2 = (D2 < keep_prob).astype(int)
  A2 = A2 * D2
  A2 = A2 / (keep_prob)

  Z3 = np.dot(W3, A2) + b3
  A3 = sigmoid(Z3)

  cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)

  return A3, cache


## Backward_propagation_with_dropout

(1) 같은 뉴런들을 shut down함
(2) A1 / keep_prob했던 것 처럼 dA1 / keep_prop할 것임 for scaling

In [1]:
def backward_propagation_with_dropout(X, Y, cache, keep_prob):

  m = X.shape[1]
  (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache

  dZ3 = A3-Y
  dW3 = (1/m) * np.dot(dZ3, A2.T)
  db3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)
  dA2 = np.dot(W3.T, dZ3)
  #step1: apply mask D2 to shut down the same neurons
  dA2 = dA2 * D2
  #step2: scale the value of neurons that haven't been shout down
  dA2 = dA2 / keep_prob

  dZ2 = np.multiply(dA2, np.int64(A2>0))
  dW2 = (1/m) * np.dot(dZ2, A1.T)
  db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

  dA1 = np.dot(W2.T, dZ2)
  dA1 = dA1 * D1
  dA1 = dA1 / keep_prob

  dZ1 = np.multiply(dA1, np.int64(A1 > 0))
  dW1 = (1/m) * np.dot(dZ1, X.T)
  db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

  gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
               "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1,
                "dZ1": dZ1, "dW1": dW1, "db1": db1}
  return gradients